## Task 2: Mushroom Classification – Project Overview

The goal of this project is to build and evaluate classification models that predict whether a mushroom is **edible** or **poisonous** based on its physical characteristics.

The dataset contains only **categorical features**, such as cap shape, gill color, odor, etc. This makes it ideal for testing classification algorithms and preprocessing techniques for categorical data.

In this notebook, we will:
1. Load and explore the dataset, including class distribution and unique values of key features.
2. Perform exploratory data analysis (EDA) using bar plots and correlation analysis (Cramér’s V).
3. Preprocess the data using appropriate encoders (OneHotEncoder or OrdinalEncoder).
4. Train and evaluate multiple classification models, including:
   - Logistic Regression
   - Decision Tree
   - Random Forest
   - Naive Bayes
   - Gradient Boosting
5. Measure performance using accuracy, precision, recall, F1 score, and confusion matrix.
6. Visualize the most important features influencing classification.
7. Analyze model performance and draw conclusions about feature importance and potential improvements.

The ultimate goal is to identify reliable patterns that distinguish edible from poisonous mushrooms with high accuracy.


## Import required libraries